# **Instructions for Object Detection with Trained Model**:

## Step 1: Open the Game
- Open the game that you want to perform detections on.

## Step 2: Identify the Game Window
- Look at the title bar of the game window. This is usually at the top of the window and contains the name of the game.

## Step 3: Update the Script
- Open the `4_yolo_opencv_detector.ipynb` file in your Python IDE (like PyCharm).
- Find the line that says `window_name = "Old School RuneScape"`.
- Replace `"Old School RuneScape"` with the name of your game's title bar. Make sure to keep the quotes around the name.

## Step 4: Run the Script
- Run all cells in the `4_yolo_opencv_detector.ipynb` file. This will start detecting objects using your trained model.

In [5]:
# Import necessary libraries
import numpy as np
import win32gui, win32ui, win32con
from PIL import Image
from time import sleep
import cv2 as cv
import os
import random

# WindowCapture class captures screenshots from the specified window
class WindowCapture:
    w = 0
    h = 0
    hwnd = None

    def __init__(self, window_name):
        # Find the window using its name
        self.hwnd = win32gui.FindWindow(None, window_name)
        if not self.hwnd:
            raise Exception('Window not found: {}'.format(window_name))

        # Get the window size
        window_rect = win32gui.GetWindowRect(self.hwnd)
        self.w = window_rect[2] - window_rect[0]
        self.h = window_rect[3] - window_rect[1]

        # Define the area to capture
        border_pixels = 8
        titlebar_pixels = 30
        self.w = self.w - (border_pixels * 2)
        self.h = self.h - titlebar_pixels - border_pixels
        self.cropped_x = border_pixels
        self.cropped_y = titlebar_pixels

    def get_screenshot(self):
        # Get a screenshot from the window
        wDC = win32gui.GetWindowDC(self.hwnd)
        dcObj = win32ui.CreateDCFromHandle(wDC)
        cDC = dcObj.CreateCompatibleDC()
        dataBitMap = win32ui.CreateBitmap()
        dataBitMap.CreateCompatibleBitmap(dcObj, self.w, self.h)
        cDC.SelectObject(dataBitMap)
        cDC.BitBlt((0, 0), (self.w, self.h), dcObj, (self.cropped_x, self.cropped_y), win32con.SRCCOPY)

        # Convert the screenshot to a numpy array
        signedIntsArray = dataBitMap.GetBitmapBits(True)
        img = np.fromstring(signedIntsArray, dtype='uint8')
        img.shape = (self.h, self.w, 4)

        # Clean up memory
        dcObj.DeleteDC()
        cDC.DeleteDC()
        win32gui.ReleaseDC(self.hwnd, wDC)
        win32gui.DeleteObject(dataBitMap.GetHandle())

        # Remove the alpha channel
        img = img[...,:3]
        img = np.ascontiguousarray(img)

        return img

    def generate_image_dataset(self):
        # Generate an image dataset for training
        if not os.path.exists("images"):
            os.mkdir("images")
        while(True):
            img = self.get_screenshot()
            im = Image.fromarray(img[..., [2, 1, 0]])
            im.save(f"./images/img_{len(os.listdir('images'))}.jpeg")
            sleep(1)

    def get_window_size(self):
        # Get the size of the window
        return (self.w, self.h)

# ImageProcessor class processes the images and performs object detection
class ImageProcessor:
    W = 0
    H = 0
    net = None
    ln = None
    classes = {}
    colors = []

    def __init__(self, img_size, cfg_file, weights_file):
        # Initialize the neural network
        np.random.seed(42)
        self.net = cv.dnn.readNetFromDarknet(cfg_file, weights_file)
        self.net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
        self.ln = self.net.getLayerNames()
        self.ln = [self.ln[i-1] for i in self.net.getUnconnectedOutLayers()]
        self.W = img_size[0]
        self.H = img_size[1]

        # Load the class labels
        with open('yolov4-tiny/obj.names', 'r') as file:
            lines = file.readlines()
        for i, line in enumerate(lines):
            self.classes[i] = line.strip()

        # Define the colors for the bounding boxes
        self.colors = [
            (0, 0, 255), 
            (0, 255, 0), 
            (255, 0, 0), 
            (255, 255, 0), 
            (255, 0, 255), 
            (0, 255, 255)
        ]

    def proccess_image(self, img):
        # Process the image and perform object detection
        blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outputs = self.net.forward(self.ln)
        outputs = np.vstack(outputs)

        coordinates = self.get_coordinates(outputs, 0.5)

        self.draw_identified_objects(img, coordinates)

        return coordinates

    def get_coordinates(self, outputs, conf):
        # Get the coordinates of the detected objects
        boxes = []
        confidences = []
        classIDs = []

        for output in outputs:
            scores = output[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > conf:
                x, y, w, h = output[:4] * np.array([self.W, self.H, self.W, self.H])
                p0 = int(x - w//2), int(y - h//2)
                boxes.append([*p0, int(w), int(h)])
                confidences.append(float(confidence))
                classIDs.append(classID)

        indices = cv.dnn.NMSBoxes(boxes, confidences, conf, conf-0.1)

        if len(indices) == 0:
            return []

        coordinates = []
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            coordinates.append({'x': x, 'y': y, 'w': w, 'h': h, 'class': classIDs[i], 'class_name': self.classes[classIDs[i]]})

        return coordinates

    def draw_identified_objects(self, img, coordinates):
        # Draw the bounding boxes and class labels on the image
        for coordinate in coordinates:
            x = coordinate['x']
            y = coordinate['y']
            w = coordinate['w']
            h = coordinate['h']
            classID = coordinate['class']

            color = self.colors[classID]

            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv.putText(img, self.classes[classID], (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv.imshow('window',  img)

# Run this cell to initiate detections using the trained model.
window_name = "Old School RuneScape"
cfg_file_name = "./yolov4-tiny/yolov4-tiny-custom.cfg"
weights_file_name = "yolov4-tiny-custom_last.weights"

wincap = WindowCapture(window_name)
improc = ImageProcessor(wincap.get_window_size(), cfg_file_name, weights_file_name)

while(True):
    # Get a screenshot from the game window
    ss = wincap.get_screenshot()

    # If 'q' is pressed on the keyboard, stop the loop
    if cv.waitKey(1) == ord('q'):
        cv.destroyAllWindows()
        break

    # Process the screenshot and perform object detection
    coordinates = improc.proccess_image(ss)

    # Print the coordinates of the detected objects
    for coordinate in coordinates:
        print(coordinate)
    print()

    # If you have limited computer resources, consider adding a sleep delay between detections.
    # sleep(0.2)

print('Finished.')

C:\Users\pkmaz\AppData\Local\Temp\ipykernel_26460\3533185580.py:32: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  img = np.fromstring(signedIntsArray, dtype='uint8')


{'x': 78, 'y': 150, 'w': 97, 'h': 92, 'class': 0, 'class_name': 'tree'}
{'x': 161, 'y': 230, 'w': 70, 'h': 63, 'class': 0, 'class_name': 'tree'}
{'x': 114, 'y': 54, 'w': 83, 'h': 97, 'class': 0, 'class_name': 'tree'}
{'x': 74, 'y': 30, 'w': 50, 'h': 68, 'class': 0, 'class_name': 'tree'}
{'x': 13, 'y': 119, 'w': 65, 'h': 54, 'class': 0, 'class_name': 'tree'}
{'x': 5, 'y': 12, 'w': 45, 'h': 68, 'class': 0, 'class_name': 'tree'}
{'x': 21, 'y': 249, 'w': 66, 'h': 50, 'class': 0, 'class_name': 'tree'}

{'x': 78, 'y': 150, 'w': 97, 'h': 92, 'class': 0, 'class_name': 'tree'}
{'x': 161, 'y': 230, 'w': 70, 'h': 63, 'class': 0, 'class_name': 'tree'}
{'x': 114, 'y': 54, 'w': 83, 'h': 97, 'class': 0, 'class_name': 'tree'}
{'x': 74, 'y': 30, 'w': 50, 'h': 68, 'class': 0, 'class_name': 'tree'}
{'x': 13, 'y': 119, 'w': 65, 'h': 54, 'class': 0, 'class_name': 'tree'}
{'x': 5, 'y': 12, 'w': 45, 'h': 68, 'class': 0, 'class_name': 'tree'}
{'x': 21, 'y': 249, 'w': 66, 'h': 50, 'class': 0, 'class_name': 'tr